## Moodle Database: Educational Data Log Analysis 
## Overview
This week you will analyse the 2019 10 Academy learners activity in the Moodle Learning Management System. The moodle LMS is a highly scalable framework, and all students activities are stored in a highly structured database.  
## Business Need
Many educational facilities such as colleges, universities, bootcamps rely on scalable and versatile Learning Management Systems. 
The Moodle LMS  is a free and open-source learning management system written in PHP and distributed under the GNU General Public License. It is used for blended learning, distance education, flipped classroom and other e-learning projects in schools, universities, workplaces and other sectors. With customizable management features, it is used to create private websites with online courses for educators and trainers to achieve learning goals. Moodle allows for extending and tailoring learning environments using community-sourced plugins.

In 2019, 10 Academy used the Moodle LMS to manage about 1000 students in their 6 months data science training. Learners, course instructors, and all admins interacted with the Moodle system for almost all the training activities. All events from these activities are logged in the moodle postgres database. 
10 Academy, like any other educational facility, is interested to understand the learners skill and knowledge development, and is interested to build models that are able to predict possible dropouts as well as classify learners into doing, well, doing ok, and struggling groups. 10 Academy is also interested in automating the process of reminding learners approaching deadlines, providing quick feedback based on their overall community engagement and performance. Moreover, given the main goal of 10 Academy training is to make students job ready, it wants to measure students' performance across many relevant metrics. 

## Task 1 - Moodle database schema understanding
Your first task is to have a clear understanding of the moodle database schema - start by studying this page. In parallel do the following 


(a). Download the database and write an SQL script to count 
    1. the number of tables 
    2. the number of records in each of the tables given in the MIT section
    3. Number of quiz submissions by hour of day
    4. Monthly usage time of learners who have confirmed and are not deleted
    5. Count of log events per user for the following verbs: ['loggedin', 'viewed', 'started', ,'submitted', 'uploaded', 'updated', 'searched', 'resumed', 'answered', 'attempted', 'abandoned']
(b). Write a python class to pull 
    1. Overall grade of learners for 
    2. Number of forum posts



In [1]:
import psycopg2 as psy
import pandas as pd

In [2]:
# Connect to an existing database

conn = psy.connect("dbname=moodle user=postgres password=admin")

In [3]:
# open a cursor to perform database operations

cur = conn.cursor()

In [4]:
# execute a command: select all from the table
cur.execute("SELECT COUNT(*) FROM information_schema.tables")

#### The number of tables

In [5]:
rows = cur.fetchall()
for items in rows:
    print("The total number of rows in the table is: ")
    print (items[0])

The total number of rows in the table is: 
642


### The number of tables where SCHEMA is "public"

In [6]:
# Specifying the column "public"
cur.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'public'" )


In [7]:
public = cur.fetchall()
for items in public:
    print("The total number of table is: ")
    print (items[0])

The total number of table is: 
448


### The number of records in each of the tables given in the MIT section

In [8]:
class IntSet(object):
    def __init__(self):
        # Creates an empty sets of integers
        self.vals = []

    def insert(self, tables):
        # Assumes tables is an integer and inserts tables into self
        # Checks if e is not in self and does not create a copy
        for table in tables:
        	cur.execute(f'select count(*) from {table}')
        	rows = cur.fetchall()
        	self.vals.append([table, rows])
        columns = ['Table', 'Number of rows']
        tables_df = pd.DataFrame(self.vals, columns = columns)
        return tables_df

In [9]:
s = IntSet()

In [10]:
the_table = (['mdl_logstore_standard_log', 'mdl_context', 'mdl_user', 'mdl_course', 'mdl_modules', 'mdl_course_modules', 'mdl_course_modules_completion', 'mdl_grade_items', 'mdl_grade_grades', 'mdl_grade_categories', 'mdl_grade_items_history', 'mdl_grade_grades_history', 'mdl_grade_categories_history', 'mdl_forum', 'mdl_forum_discussions', 'mdl_forum_posts'])

In [11]:
s.insert(the_table)

,Table,Number of rows
0,mdl_logstore_standard_log,"[(417554,)]"
1,mdl_context,"[(4359,)]"
2,mdl_user,"[(1052,)]"
3,mdl_course,"[(15,)]"
4,mdl_modules,"[(26,)]"
5,mdl_course_modules,"[(290,)]"
6,mdl_course_modules_completion,"[(4483,)]"
7,mdl_grade_items,"[(113,)]"
8,mdl_grade_grades,"[(3643,)]"
9,mdl_grade_categories,"[(16,)]"


### Number of quiz submissions by hour of day

In [12]:
cur.execute("select count(id), EXTRACT(HOUR FROM to_timestamp(timecreated)) from mdl_logstore_standard_log where action='submitted' AND component='mod_quiz' group by EXTRACT(HOUR FROM to_timestamp(timecreated))")

data = cur.fetchall()

daily_quiz_per_hour = pd.DataFrame(data, columns = ['Number of Quiz Submissions', 'Hour of day']).set_index('Hour of day')
daily_quiz_per_hour

,Number of Quiz Submissions
Hour of day,
0.0,29
1.0,7
2.0,14
3.0,10
4.0,18
5.0,17
6.0,34
7.0,46
8.0,69


### Monthly usage time of learners who have confirmed and are not deleted

In [13]:
def log_events(events):
    df = pd.DataFrame(columns = ['userid'])
    for event in events:
        cur.execute("Select userid, count(*) from mdl_logstore_standard_log where action = '{e}' group by userid".format(e=event))
        rows = cur.fetchall()
        df1 = pd.DataFrame(rows, columns = ['userid', event])
        df = df.merge(df1, on = ['userid'], how = 'outer').fillna(0)
        
    events_log = df.copy().set_index('userid')
    return events_log

In [14]:
the_table = (['loggedin', 'viewed', 'started', 'submitted', 'uploaded', 'updated', 'searched', 'resumed', 'answered', 'attempted', 'abandoned'])

In [15]:
log_events(the_table)

,loggedin,viewed,started,submitted,uploaded,updated,searched,resumed,answered,attempted,abandoned
userid,,,,,,,,,,,
1,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,169.0,2492,14.0,21.0,21.0,2551.0,0.0,0.0,0.0,0,5.0
3,107.0,4041,17.0,2.0,6.0,1467.0,0.0,4.0,0.0,0,0.0
5,54.0,694,11.0,5.0,1.0,19.0,0.0,2.0,2.0,0,0.0
7,3.0,81,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1046,0.0,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1049,0.0,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1050,0.0,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


### Python class to pull Overall grade of learners for Number of forum posts


In [16]:
import numpy as np

In [17]:
class GradeForums:
    def grades():
        conn = psy.connect(host="localhost", port = 5432, database="moodle", user="postgres", password="admin")
        grade_df = pd.read_sql_query('select * from "mdl_grade_grades"',con=conn)
        average_grade = np.mean(grade_df['finalgrade'])
        print('The overall grade of learners is:', average_grade)
    def forums():
        cur.execute('select count(*) from mdl_forum_posts')
        rows = cur.fetchall()
        df = pd.DataFrame(rows, columns = ['Number of forum posts'])
        return df

In [18]:
gf = GradeForums

In [19]:
gf.grades()

The overall grade of learners is: 107.37171772685176


In [20]:
gf.forums()

,Number of forum posts
0,131
